<a href="https://colab.research.google.com/github/sayed6201/nlp_20newsgroups_exercise/blob/main/exercise_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Imports**

In [ ]:
import sklearn as sklearn
from sklearn.datasets import fetch_20newsgroups
from pprint import pprint
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer


# **Data Importing**

In [ ]:
newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')

In [ ]:
pprint(list(newsgroups_train.target_names))
pprint(dir(newsgroups_train))
pprint(newsgroups_train.target)

pprint(len(newsgroups_train.data))
pprint(len(newsgroups_test.data))

x_train, y_train = newsgroups_train.data, newsgroups_train.target
x_test, y_test = newsgroups_test.data, newsgroups_test.target

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']
['DESCR', 'data', 'filenames', 'target', 'target_names']
array([7, 4, 4, ..., 3, 1, 8])
11314
7532


In [ ]:
x_train[0]

"From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n   ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n"

# **Data Preprocessing**


In [ ]:
countVect = CountVectorizer()
x_train_countV = countVect.fit_transform(x_train)
print(f"Shape of Term Frequency Matrix: {x_train_countV.shape}")

tfidfTrans = TfidfTransformer()
x_train_tfidf = tfidfTrans.fit_transform(x_train_countV)
print(f"Shape of TFIDF Matrix: {x_train_tfidf.shape}")

Shape of Term Frequency Matrix: (11314, 130107)
Shape of TFIDF Matrix: (11314, 130107)
